In [1]:
# Import dependencies
# !pip install wrds # required only first time when this file is run on a system
import wrds
import pandas as pd

# !pip install cpi # required first time on a system
import cpi

/Users/snehakumari/anaconda3/envs/Data/lib/python3.7/site-packages/cpi/__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


## Setting up connection

In [2]:
db = wrds.Connection(wrds_username='ab1234')
db.create_pgpass_file()
# run this to close connection
# db.close()

Loading library list...
Done


Enter your WRDS username [snehakumari]: ab1234
Enter your password: ·············


In [3]:
db = wrds.Connection(wrds_username='ab1234') ##run this if you get logged out at any stage -- change this to your username

Loading library list...
Done


List of db funtionc available:
db.close()
db.connection()
db.describe_table()
db.get_table()
db.list_tables()
db.raw_sql()
db.get_row_count()
db.list_libraries()

help(db.get_table)
help(db.raw_sql)

db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=10)

db.raw_sql('select date,dji from djones.djdaily LIMIT 10;', date_cols=['date'])

## Update sp500 constituents list

db.describe_table('compd','idxcst_his')

#### gvkeyx for S&P500 index is 000003, so we will filter out that in our query
Finding the gvkeyx for sp500 index -- it is located in this
https://wrds-www.wharton.upenn.edu/search_variables/
Query to locate the same if we need to demonstrate:
index_names = db.raw_sql("select * from compd.idx_index")
index_names.head()

### Collecting data for Cash Flow, Income & Balance sheet items from wrds server

db.describe_table('compd','funda')

In [3]:
cf_raw = db.raw_sql("""
        SELECT gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,curcd,fyr,stalt,aoloch,apalch,aqc,capx,chech,dlcch,dltis,dltr,dpc,dv,esubc,exre,fiao,fincf,fopo,ibc,intpn,invch,ivaco,ivch,ivncf,ivstch,oancf,prstkc,recch,siv,sppe,sppiv,sstk,txach,txdc,txpd,xidoc,exchg,costat 
        FROM compd.funda where gvkey = '001690'
        """)

In [4]:
is_raw = db.raw_sql("""
        SELECT gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,curcd,fyr,stalt,cogs,cstke,dp,dvp,ib,ibadj,ibcom,mii,ni,niadj,nopi,oiadp,oibdp,pi,sale,spi,txt,xido,xint,xrd,xsga,exchg,costat
        FROM compd.funda where gvkey = '001690'
        """)

In [5]:
bs_raw = db.raw_sql("""
        SELECT gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,curcd,curncd,fyr,stalt,aco,act,ao,ap,at,ceq,che,dlc,dltt,intan,invt,ivaeq,ivao,lco,lct,lo,lt,mib,ppent,pstk,rect,seq,txditc,txp,exchg
        FROM compd.funda where gvkey = '001690'
        """)

In [6]:
cf_raw.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,siv,sppe,sppiv,sstk,txach,txdc,txpd,xidoc,exchg,costat
0,001690,1980-09-30,1980.0,INDL,C,D,STD,AAPL,037833100,APPLE INC,...,0.0,0.0,NaN,NaN,NaN,0.747,NaN,0.0,14.0,A
1,001690,1981-09-30,1981.0,INDL,C,D,STD,AAPL,037833100,APPLE INC,...,0.0,NaN,NaN,NaN,NaN,4.311,NaN,0.0,14.0,A
2,001690,1982-09-30,1982.0,INDL,C,D,STD,AAPL,037833100,APPLE INC,...,0.0,NaN,NaN,NaN,NaN,7.625,NaN,0.0,14.0,A
3,001690,1983-09-30,1983.0,INDL,C,D,STD,AAPL,037833100,APPLE INC,...,0.0,NaN,NaN,NaN,NaN,35.697,NaN,0.0,14.0,A
4,001690,1984-09-30,1984.0,INDL,C,D,STD,AAPL,037833100,APPLE INC,...,0.0,NaN,NaN,23.242,NaN,20.453,NaN,0.0,14.0,A


##### Clean the files to achieve unique gvkey/year pair. The cleaning filters are the same for the three. 

Following filters are to be applied: 
    
    1) From wrds interface: popsrc = D, consol = C, datafmt = STD
    
    2) From logic: curcd = USD, indfmt != FS

In [7]:
#Filtering cash flow

cf_filtered = cf_raw[cf_raw['curcd']=='USD']
cf_filtered = cf_filtered[cf_filtered['indfmt']!='FS']
cf_filtered = cf_filtered[cf_filtered['datafmt']=='STD']
cf_filtered = cf_filtered[cf_filtered['popsrc']=='D']
cf_filtered = cf_filtered[cf_filtered['consol']=='C']

In [8]:
#Filtering balance sheet

bs_filtered = bs_raw[bs_raw['curcd']=='USD']
bs_filtered = bs_filtered[bs_filtered['indfmt']!='FS']
bs_filtered = bs_filtered[bs_filtered['datafmt']=='STD']
bs_filtered = bs_filtered[bs_filtered['popsrc']=='D']
bs_filtered = bs_filtered[bs_filtered['consol']=='C']

In [9]:
#Filtering income statement

is_filtered = is_raw[is_raw['curcd']=='USD']
is_filtered = is_filtered[is_filtered['indfmt']!='FS']
is_filtered = is_filtered[is_filtered['datafmt']=='STD']
is_filtered = is_filtered[is_filtered['popsrc']=='D']
is_filtered = is_filtered[is_filtered['consol']=='C']

##### One line code for the section below after adding year line --however slightly slow for big dataset

cf = cf_filtered[cf_lastreport = cf_filtered.groupby(['gvkey', 'year'])['datadate'].transform(max)==cf_filtered['datadate']]

In [10]:
# Adding year from datadate to extract last report
cf_filtered['year'] = pd.DatetimeIndex(cf_filtered['datadate']).year

# filtering rows which has the lastreport for that year to get a unique pair of gvkey/year
cf_grouped = cf_filtered.groupby(['gvkey', 'year']).agg({'datadate':'max'})
cf_grouped = cf_grouped.reset_index()
cf_grouped = cf_grouped.rename(columns={'datadate':'lastreport'})
cf = pd.merge(cf_filtered, cf_grouped, how = 'left', on = ['gvkey', 'year'])
cf = cf[cf['datadate'] == cf['lastreport']]

#Check if it yields to unique pair (should return (0,))
cf[cf.duplicated(['gvkey', 'year'], keep=False)].shape

(0, 50)

In [11]:
# Adding year from datadate to extract last report
bs_filtered['year'] = pd.DatetimeIndex(bs_filtered['datadate']).year

# Filtering rows which has the lastreport for that year to get a unique pair of gvkey/year
bs_grouped = bs_filtered.groupby(['gvkey', 'year']).agg({'datadate':'max'})
bs_grouped = bs_grouped.reset_index()
bs_grouped = bs_grouped.rename(columns={'datadate':'lastreport'})
bs = pd.merge(bs_filtered, bs_grouped, how = 'left', on = ['gvkey', 'year'])
bs = bs[bs['datadate'] == bs['lastreport']]

#Check if it yields to unique pair (should return (0,))
bs[bs.duplicated(['gvkey', 'year'], keep=False)].shape

(0, 41)

In [12]:
# Adding year from datadate to extract last report
is_filtered['year'] = pd.DatetimeIndex(is_filtered['datadate']).year

# filtering rows which has the lastreport for that year to get a unique pair of gvkey/year
is_grouped = is_filtered.groupby(['gvkey', 'year']).agg({'datadate':'max'})
is_grouped = is_grouped.reset_index()
is_grouped = is_grouped.rename(columns={'datadate':'lastreport'})
is_d = pd.merge(is_filtered, is_grouped, how = 'left', on = ['gvkey', 'year'])
is_d = is_d[is_d['datadate'] == is_d['lastreport']]

#Check if it yields to unique pair (should return (0,))
is_d[is_d.duplicated(['gvkey', 'year'], keep=False)].shape

(0, 38)

In [15]:
### To save the files for export
##### just change this cell to code and run
cf_raw.to_csv('cf_raw.csv')
is_raw.to_csv('is_raw.csv')
bs_raw.to_csv('bs_raw.csv')
cf.to_csv('cf_final.csv')
is_d.to_csv('is_final.csv')
bs.to_csv('bs_final.csv')
index_sp500.to_csv('sp500const.csv')

## Collapse data by year and sequence it as desired

In [15]:
bs_nominal = bs.groupby('year')['che','rect','invt','aco','act','ppent','ivaeq','ivao','intan','ao','at','dlc','ap','txp','lco','lct','dltt','lo','txditc','mib','lt','pstk','ceq','seq'].sum()
bs_nominal['firm_count']= bs.groupby('year').gvkey.nunique()

In [16]:
is_nominal = is_d.groupby('year')['sale','cogs','xsga','oibdp','dp','oiadp','xint','nopi','spi','pi','txt','mii','ib','dvp','ibcom','cstke','ibadj','xido','niadj','ni','xrd'].sum()
is_nominal['firm_count']= is_d.groupby('year').gvkey.nunique()

In [17]:
cf_nominal = cf.groupby('year')['ibc','dpc','xidoc','txdc','esubc','sppiv','fopo','recch','invch','apalch','txach','aoloch','oancf','ivch','siv','ivstch','capx','sppe','aqc','ivaco','ivncf','sstk','prstkc','dv','dltis','dltr','dlcch','fiao','fincf','exre','chech'].sum()
cf_nominal['firm_count']= cf.groupby('year').gvkey.nunique()

In [18]:
consolidated = pd.concat([is_nominal['ni'],cf_nominal['dpc'],cf_nominal['oancf'],cf_nominal['capx'],cf_nominal['aqc'],cf_nominal['sppe'],cf_nominal['ivch'],cf_nominal['siv'],cf_nominal['ivstch'],cf_nominal['ivaco'],cf_nominal['ivncf'],cf_nominal['dltis'],cf_nominal['dltr'],cf_nominal['dlcch'],cf_nominal['dv'],cf_nominal['sstk'],cf_nominal['prstkc'],cf_nominal['fiao'],cf_nominal['fincf'],cf_nominal['exre'],cf_nominal['chech'],cf_nominal['firm_count']],axis=1)
# var = oancf+ivncf+fincf+exre
consolidated['var'] = cf_nominal['oancf']+cf_nominal['ivncf']+cf_nominal['fincf']+cf_nominal['exre']

In [20]:
# Get year in data
bs_nominal['year'] = bs_nominal.index 
cf_nominal['year'] = cf_nominal.index
is_nominal['year'] = is_nominal.index
consolidated['year'] = consolidated.index

NameError: name 'consolidated_real' is not defined

In [22]:
consolidated

,ni,dpc,oancf,capx,aqc,sppe,ivch,siv,ivstch,ivaco,...,dv,sstk,prstkc,fiao,fincf,exre,chech,firm_count,var,year
year,,,,,,,,,,,,,,,,,,,,,
1980,11.698,1.377,0.000,4.878,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1,0.000,1980
1981,39.420,8.590,0.000,24.529,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1,0.000,1981
1982,61.306,16.556,0.000,26.470,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1,0.000,1982
1983,76.714,22.440,0.000,52.666,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1,0.000,1983
1984,64.055,37.963,0.000,39.614,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,23.242,0.000,0.000,0.000,0.0,-28.396,1,0.000,1984
1985,61.223,41.841,0.000,54.064,0.0,12.744,0.0,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.0,222.125,1,0.000,1985
1986,153.963,51.075,0.000,66.629,0.0,0.000,0.0,0.0,0.000,0.000,...,0.000,0.000,54.505,0.000,0.000,0.0,239.202,1,0.000,1986
1987,217.496,70.516,0.000,86.127,0.0,0.000,0.0,0.0,0.000,0.000,...,15.234,0.000,154.136,0.000,0.000,0.0,-11.121,1,0.000,1987
1988,400.258,77.677,294.440,144.001,0.0,0.000,0.0,0.0,0.000,-42.103,...,39.646,0.000,299.425,83.487,-127.713,0.0,-19.377,1,-19.377,1988


In [23]:
#Tranpose the dataframes before exporting
cf_nominal = cf_nominal.transpose() 
is_nominal = is_nominal.transpose()
bs_nominal = bs_nominal.transpose()
consolidated = consolidated.transpose()

In [24]:
# Get codes in data
bs_nominal['code'] = bs_nominal.index 
is_nominal['code'] = is_nominal.index 
cf_nominal['code'] = cf_nominal.index
consolidated['code'] = consolidated.index

In [25]:
#import the code file
codelist = pd.read_csv("python/item_codes.csv")

#insert the item list for codes
cf_nominal = pd.merge(cf_nominal, codelist, how = 'left', on = ['code'])
bs_nominal = pd.merge(bs_nominal, codelist, how = 'left', on = ['code'])
is_nominal = pd.merge(is_nominal, codelist, how = 'left', on = ['code'])
consolidated = pd.merge(consolidated, codelist, how = 'left', on = ['code'])

In [26]:
#remove nan value years from consolidated file
consolidated_nominal = consolidated.iloc[:,26:] #remove years till 1990

# Create the groups of year values
consolidated_nominal['1990-2002']= consolidated_nominal.iloc[:,:13].sum(axis=1)
consolidated_nominal['2003-2015']= consolidated_nominal.iloc[:,13:26].sum(axis=1)
consolidated_nominal['2016-2018']= consolidated_nominal.iloc[:,26:29].sum(axis=1)
# calculating % age of NI for each year range
consolidated_nominal['1990-2002%']= consolidated_nominal['1990-2002'].div(consolidated_nominal.iloc[0]['1990-2002'])
consolidated_nominal['2003-2015%']= consolidated_nominal['2003-2015'].div(consolidated_nominal.iloc[0]['2003-2015'])
consolidated_nominal['2016-2018%']= consolidated_nominal['2016-2018'].div(consolidated_nominal.iloc[0]['2016-2018'])


In [27]:
#add code and code descriptions in the start and freeze columns
cf_nominal.insert(loc=0, column='Values', value=cf_nominal['item'])
bs_nominal.insert(loc=0, column='Values', value=bs_nominal['item'])
is_nominal.insert(loc=0, column='Values', value=is_nominal['item'])
consolidated_nominal.insert(loc=0, column='Values', value=consolidated_nominal['item'])

In [28]:
consolidated.insert(loc=0, column='Values', value=consolidated['item'])

In [29]:
# Write all this to excel and format
import xlsxwriter

writer = pd.ExcelWriter('consolidated_financials_aapl.xlsx', engine='xlsxwriter')

cf_nominal.to_excel(writer, index=False, sheet_name='CashFlow')
is_nominal.to_excel(writer, index=False, sheet_name='IncomeSt')
bs_nominal.to_excel(writer, index=False, sheet_name='BalanceSheet')
consolidated.to_excel(writer, index=False, sheet_name='Consolidated')
workbook = writer.book

#Format structures
#format structures defined here
money_fmt = workbook.add_format({'num_format': '$#,##0'})
total_fmt = workbook.add_format({'align': 'right','bottom':1, 'top':1})
highlight_fmt = workbook.add_format({'bg_color': 'cyan','bold': True,'font_size':13, 'num_format': '$#,##0'})
bold_italic = workbook.add_format({'bold': True, 'italic': True})
minus_fmt = workbook.add_format({'font_color': 'red'})
plus_fmt = workbook.add_format({'font_color': 'green'})
percent_fmt = workbook.add_format({'num_format': '0.0%'})
header_format = workbook.add_format({'bold': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC',
                                     'border': 1})

#specify formatting for cashflow
worksheet = writer.sheets['CashFlow']
# worksheet.set_zoom(110)
worksheet.freeze_panes(1, 1)
worksheet.set_column(0, 40)
worksheet.set_row(13, None, highlight_fmt)
worksheet.set_row(21, None, highlight_fmt)
worksheet.set_row(29, None, highlight_fmt)
worksheet.set_row(30, None, bold_italic)
worksheet.set_row(31, None, bold_italic)
worksheet.set_column('A:BC',15, money_fmt)

#specify formatting for balance sheet
worksheet = writer.sheets['BalanceSheet']
# worksheet.set_zoom(110)
worksheet.freeze_panes(1, 1)
worksheet.set_column(0, 45)
worksheet.set_row(5, None, highlight_fmt)
worksheet.set_row(11, None, highlight_fmt)
worksheet.set_row(16, None, highlight_fmt)
worksheet.set_row(21, None, highlight_fmt)
worksheet.set_row(24, None, highlight_fmt)
worksheet.set_column('A:BC',15, money_fmt)

#specify formatting for Income Statement
worksheet = writer.sheets['IncomeSt']
# worksheet.set_zoom(110)
worksheet.freeze_panes(1, 1)
worksheet.set_column(0, 60)
worksheet.set_row(1, None, plus_fmt)
worksheet.set_row(2, None, minus_fmt)
worksheet.set_row(3, None, minus_fmt)
worksheet.set_row(4, None, highlight_fmt)
worksheet.set_row(5, None, minus_fmt)
worksheet.set_row(6, None, highlight_fmt)
worksheet.set_row(7, None, minus_fmt)
worksheet.set_row(8, None, plus_fmt)
worksheet.set_row(9, None, plus_fmt)
worksheet.set_row(10, None, highlight_fmt)
worksheet.set_row(11, None, minus_fmt)
worksheet.set_row(12, None, minus_fmt)
worksheet.set_row(13, None, highlight_fmt)
worksheet.set_row(14, None, minus_fmt)
worksheet.set_row(15, None, highlight_fmt)
worksheet.set_row(16, None, plus_fmt)
worksheet.set_row(17, None, highlight_fmt)
worksheet.set_row(18, None, plus_fmt)
worksheet.set_row(19, None, highlight_fmt)
worksheet.set_row(20, None, bold_italic)
worksheet.set_row(21, None, bold_italic)
worksheet.set_column('A:BC',15, money_fmt)


#specify formatting for Income Statement
worksheet = writer.sheets['Consolidated']
worksheet.freeze_panes(1, 1)
worksheet.set_column(0, 35)
# worksheet.set_zoom(110)
worksheet.set_row(3, None, highlight_fmt)
worksheet.set_row(11, None, highlight_fmt)
worksheet.set_row(19, None, highlight_fmt)
worksheet.set_row(20, None, highlight_fmt)
worksheet.set_column('A:AD',15, money_fmt)
worksheet.set_column('AF:AI',15, money_fmt)
worksheet.set_column('AF:AI',15, money_fmt)
worksheet.set_column('AI:AL',12, percent_fmt)

writer.save()


In [50]:
#Tranpose it again before carrying on further analyses
cf_nominal = cf_nominal.transpose() 
is_nominal = is_nominal.transpose()
bs_nominal = bs_nominal.transpose()

# Save files to single file
# !pip install xlsxwriter #required to run only first time
import xlsxwriter

writer = pd.ExcelWriter('consolidated_financials_sp500_nominal.xlsx', engine='xlsxwriter')

cf_nominal.to_excel(writer, index=False, sheet_name='CashFlow')
is_nominal.to_excel(writer, index=False, sheet_name='IncomeSt')
bs_nominal.to_excel(writer, index=False, sheet_name='BalanceSheet')
consolidated_nominal.to_excel(writer, index=False, sheet_name='Consolidated')
workbook = writer.book
writer.save()

### WIP : run cpi index to get the real value as per 2009 data

In [55]:
#create a column of year to use for formula
bs_nominal['year'] = bs_nominal.index 
cf_nominal['year'] = cf_nominal.index
is_nominal['year'] = is_nominal.index


In [ ]:
bs_nominal['cpi_2009'] = None
cf_nominal['cpi_2009'] = None
is_nominal['cpi_2009'] = None

In [52]:
#Define a function to generate the inflation index : cpi.inflate(dollar value, base year, to year)
# change base year to the year desired
def return_cpi(row_year):
    return cpi.inflate(value=1,year_or_month=2009,to=row_year.year)

In [56]:
bs_nominal
# Remove the value row before proceeding with analyses -- 
# ideally make a copy of those nominal files before processing for output


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,year,cpi_2009
Values,cash and short-term investments,receivables - total,inventories - total,current assets - other,current aseets - total,"property,plant,and equipment - total (net)",investments and advances - equity method,investments and advances - other,intangibles,asset - other,...,deferred taxes and investment tax credit,minority interest,liabilities - total,preferred stock - carrying value,common equity - total,stockholders' equity - total,NaN,NaN,Values,None
1964,27442.3,41708.5,42793.7,896.172,115863,169132,9057.45,4549.28,1588.05,4774.57,...,5183.31,2972.95,121440,6600.88,122015,128740,426,1964,1964,None
1965,28193.6,47960.4,48008.4,1357.76,128489,186034,9839.83,5043.46,1998.9,5120.81,...,5976.59,3296.94,137842,6779.38,140289,147303,440,1965,1965,None
1966,25695.6,55323.2,55282.7,1724.49,141449,211184,11295.5,5790.97,2355.53,5044.4,...,6444.65,3640.1,161560,6947.44,189001,196119,453,1966,1966,None
1967,26202.2,61041.5,60625.8,2133.74,153415,237099,12074,6993.33,2989.27,5811.2,...,7594.29,4380.48,184443,7801.56,210449,218399,458,1967,1967,None
1968,29747.7,68362.1,65591.8,3469.66,170696,263296,15342.2,7384.91,3460.07,7096.32,...,10713.1,4942.24,214578,8507.7,231513,240007,472,1968,1968,None
1969,27345.3,76818.3,74110.7,4154.23,186359,292487,17915.7,9708.26,4584.7,7467.86,...,13035.3,5056.66,245787,9422.52,256995,264238,481,1969,1969,None
1970,26289.7,81831,81058.5,4903.08,198596,322641,20484.5,11129.2,5907.24,9215.46,...,15038.2,5553.1,280105,11212.1,272006,281355,484,1970,1970,None
1971,33444.1,85679.1,84640.8,6051.25,212116,347663,23414.3,10480.2,6348.59,10973.9,...,17196.2,5669,305532,13046.8,291173,302283,483,1971,1971,None
1972,37906.1,94044.4,89335.9,6714.66,231105,375827,25001.6,10606.2,6686.95,11315.5,...,19807,5927.37,332870,14991.6,311640,325041,485,1972,1972,None


In [53]:
# apply this function to our dataframes
bs_nominal['cpi_2009'] = bs_nominal.apply(return_cpi, axis=1)
cf_nominal['cpi_2009'] = cf_nominal.apply(return_cpi, axis=1)
is_nominal['cpi_2009'] = is_nominal.apply(return_cpi, axis=1)

TypeError: ('Years can only be converted to other years. Months only to other months.', 'occurred at index Values')

In [53]:
bs_real = bs_nominal.div(bs_nominal['cpi_2009'], axis = 'index')
bs_real['firm_count']=bs_nominal['firm_count']
bs_real['year']=bs_nominal['year']
bs_real['cpi_2009']= bs_nominal['cpi_2009']

In [54]:
is_real = is_nominal.div(is_nominal['cpi_2009'], axis = 'index')
is_real['firm_count']=is_nominal['firm_count']
is_real['year']=is_nominal['year']
is_real['cpi_2009']= is_nominal['cpi_2009']

In [55]:
cf_real = cf_nominal.div(cf_nominal['cpi_2009'], axis = 'index')
cf_real['firm_count']=cf_nominal['firm_count']
cf_real['year']=cf_nominal['year']
cf_real['cpi_2009']= cf_nominal['cpi_2009']

In [56]:
consolidated_real = pd.concat([is_real['ni'],cf_real['dpc'],cf_real['oancf'],cf_real['capx'],cf_real['aqc'],cf_real['sppe'],cf_real['ivch'],cf_real['siv'],cf_real['ivstch'],cf_real['ivaco'],cf_real['ivncf'],cf_real['dltis'],cf_real['dltr'],cf_real['dlcch'],cf_real['dv'],cf_real['sstk'],cf_real['prstkc'],cf_real['fiao'],cf_real['fincf'],cf_real['exre'],cf_real['chech'],cf_real['firm_count']],axis=1)
# var = oancf+ivncf+fincf+exre
consolidated_real['var'] = cf_real['oancf']+cf_real['ivncf']+cf_real['fincf']+cf_real['exre']

In [57]:
cf_real = cf_real.transpose() 
is_real = is_real.transpose()
bs_real = bs_real.transpose()
consolidated_real=consolidated_real.transpose()

In [58]:
bs_real['code'] = bs_real.index 
is_real['code'] = is_real.index 
cf_real['code'] = cf_real.index
consolidated_real['code'] = consolidated_real.index

In [59]:
#import the code file
codelist = pd.read_csv("item_codes.csv")

In [61]:
#insert the item list for codes
cf_real = pd.merge(cf_real, codelist, how = 'left', on = ['code'])
bs_real = pd.merge(bs_real, codelist, how = 'left', on = ['code'])
is_real = pd.merge(is_real, codelist, how = 'left', on = ['code'])
consolidated_real = pd.merge(consolidated_real, codelist, how = 'left', on = ['code'])

In [63]:
consolidated_real_new = consolidated_real.iloc[:,26:] #remove years till 1990
# Create the groups of year values
consolidated_real_new['1990-2002']= consolidated_real_new.iloc[:,:13].sum(axis=1)
consolidated_real_new['2003-2015']= consolidated_real_new.iloc[:,13:26].sum(axis=1)
consolidated_real_new['2016-2018']= consolidated_real_new.iloc[:,26:29].sum(axis=1)
# calculating % age of NI for each year range
consolidated_real_new['1990-2002%']= consolidated_real_new['1990-2002'].div(consolidated_real_new.iloc[0]['1990-2002'])
consolidated_real_new['2003-2015%']= consolidated_real_new['2003-2015'].div(consolidated_real_new.iloc[0]['2003-2015'])
consolidated_real_new['2016-2018%']= consolidated_real_new['2016-2018'].div(consolidated_real_new.iloc[0]['2016-2018'])

In [ ]:
cf_real.insert(loc=0, column='Values', value=cf_real['item'])
bs_real.insert(loc=0, column='Values', value=bs_real['item'])
is_real.insert(loc=0, column='Values', value=is_real['item'])
consolidated_real_new.insert(loc=0, column='Values', value=consolidated_real_new['item'])

### Create the output in excel and format

In [76]:
writer = pd.ExcelWriter('consolidated_financials_sp500_real.xlsx', engine='xlsxwriter')
cf_real.to_excel(writer, index=False, sheet_name='CashFlow')
bs_real.to_excel(writer, index=False, sheet_name='BalanceSheet')
is_real.to_excel(writer, index=False, sheet_name='IncomeStatement')
consolidated_real_new.to_excel(writer, index=False, sheet_name='Consolidated')
workbook = writer.book

In [77]:
#format structures defined here
money_fmt = workbook.add_format({'num_format': '$#,##0'})
total_fmt = workbook.add_format({'align': 'right','bottom':1, 'top':1})
highlight_fmt = workbook.add_format({'bg_color': 'cyan','bold': True,'font_size':13, 'num_format': '$#,##0'})
bold_italic = workbook.add_format({'bold': True, 'italic': True})
minus_fmt = workbook.add_format({'font_color': 'red'})
plus_fmt = workbook.add_format({'font_color': 'green'})
percent_fmt = workbook.add_format({'num_format': '0.0%'})

In [78]:
#specify formatting for cashflow
worksheet = writer.sheets['CashFlow']
worksheet.set_zoom(110)
worksheet.set_row(13, None, highlight_fmt)
worksheet.set_row(21, None, highlight_fmt)
worksheet.set_row(29, None, highlight_fmt)
worksheet.set_row(30, None, bold_italic)
worksheet.set_row(31, None, bold_italic)
worksheet.set_column('A:BC',15, money_fmt)

0

In [79]:
#specify formatting for balance sheet
worksheet = writer.sheets['BalanceSheet']
worksheet.set_zoom(110)
worksheet.set_row(5, None, highlight_fmt)
worksheet.set_row(11, None, highlight_fmt)
worksheet.set_row(16, None, highlight_fmt)
worksheet.set_row(21, None, highlight_fmt)
worksheet.set_row(24, None, highlight_fmt)
worksheet.set_column('A:BC',15, money_fmt)

0

In [80]:
#specify formatting for Income Statement
worksheet = writer.sheets['IncomeStatement']
worksheet.set_zoom(110)
worksheet.set_row(1, None, plus_fmt)
worksheet.set_row(2, None, minus_fmt)
worksheet.set_row(3, None, minus_fmt)
worksheet.set_row(4, None, highlight_fmt)
worksheet.set_row(5, None, minus_fmt)
worksheet.set_row(6, None, highlight_fmt)
worksheet.set_row(7, None, minus_fmt)
worksheet.set_row(8, None, plus_fmt)
worksheet.set_row(9, None, plus_fmt)
worksheet.set_row(10, None, highlight_fmt)
worksheet.set_row(11, None, minus_fmt)
worksheet.set_row(12, None, minus_fmt)
worksheet.set_row(13, None, highlight_fmt)
worksheet.set_row(14, None, minus_fmt)
worksheet.set_row(15, None, highlight_fmt)
worksheet.set_row(16, None, plus_fmt)
worksheet.set_row(17, None, highlight_fmt)
worksheet.set_row(18, None, plus_fmt)
worksheet.set_row(19, None, highlight_fmt)
worksheet.set_row(20, None, bold_italic)
worksheet.set_row(21, None, bold_italic)
worksheet.set_column('A:BC',15, money_fmt)

0

In [82]:
worksheet = writer.sheets['Consolidated']
worksheet.set_zoom(110)
worksheet.set_row(3, None, highlight_fmt)
worksheet.set_row(11, None, highlight_fmt)
worksheet.set_row(19, None, highlight_fmt)
worksheet.set_row(20, None, highlight_fmt)
worksheet.set_column('A:AD',15, money_fmt)
worksheet.set_column('AF:AI',15, money_fmt)
worksheet.set_column('AF:AI',15, money_fmt)
worksheet.set_column('AI:AL',12, percent_fmt)

0

In [83]:
writer.save()